In [1]:
import * as torch from "./index.ts";

const a = new torch.Tensor([1, 2, 3], [3, 1], true);
a._name = "a_tensor";
const b = new torch.Tensor([4, 5, 6], [3, 1], true);
b._name = "b_tensor";
const c = torch.add(a, b);
c._name = "c_tensor";
console.log("a:", a.toString());
console.log("b:", b.toString());
console.log("c:", c.toString());

a: Tensor(shape=[3,1])
b: Tensor(shape=[3,1])
c: Tensor(shape=[3,1])


In [2]:
const MN = new torch.Tensor(
  [
    2, 3,
    3, 7
  ]
, [2, 2])
MN.toString()

"Tensor(shape=[2,2])"

In [3]:
c.requires_grad

true

In [4]:
const d = torch.mul(a, c);
d._name = "d_tensor";
const e = torch.matmul(d, b.transpose());
e._name = "e_tensor";

"e_tensor"

In [ ]:
d.backward(new torch.Tensor([0.001, 0.002, 0.003], [3, 1]));
// e.backward(new torch.Tensor([0.2, 0.8, 0.7], [1, 3]));

TypeError: b.size is not a function

In [6]:
function logPrevs(tensor: torch.Tensor, indent: string = "") {
  if (tensor._prev) {
    for (let i = 0; i < tensor._prev.length; i++) {
      const prev = tensor._prev[i];
      console.log(`${indent}Prev Tensor: ${prev.toString()} (Name: ${prev?._name}), Grad: ${prev._grad ? prev._grad.toString() : "None"}`);
      logPrevs(prev, indent + "|  ");
    }
  }
}

function logTree(tensor: torch.Tensor) {
  console.log("Gradient computation graph:");
  console.log(`Tensor: ${tensor.toString()} (Name: ${tensor?._name})`);
  logPrevs(tensor, "|  ");
}

logTree(d);

Gradient computation graph:
Tensor: Tensor(shape=[3,1]) (Name: d_tensor)
|  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: None
|  Prev Tensor: Tensor(shape=[3,1]) (Name: c_tensor), Grad: None
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: None
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: b_tensor), Grad: None


In [7]:
logTree(e);

Gradient computation graph:
Tensor: Tensor(shape=[3,3]) (Name: e_tensor)
|  Prev Tensor: Tensor(shape=[3,1]) (Name: d_tensor), Grad: 0.0010000000474974513,0.0020000000949949026,0.003000000026077032
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: None
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: c_tensor), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[3,1]) (Name: b_tensor), Grad: None
|  Prev Tensor: Tensor(shape=[1,3]) (Name: undefined), Grad: None


## Simple Neural Network

In [8]:
let x = new torch.Tensor([7, 8, 9,5],[1,4]);
x._name = "Input (x)"

let W_1 = torch.Tensor.randn([4,8], true);
W_1._name = "W_1"
let B_1 = torch.Tensor.randn([1,8], true);
B_1._name = "W_2"

console.log(x.shape, W_1.shape, B_1.shape)

[ 1, 4 ] [ 4, 8 ] [ 1, 8 ]


In [9]:
let h_1 = torch.add(torch.matmul(x, W_1), B_1);
h_1._name = "h_1"

let W_2 = torch.Tensor.randn([8,2], true);
W_2._name = "W_2"
let B_2 = torch.Tensor.randn([1,2], true);
B_2._name = "B_2"

console.log(h_1.shape, W_2.shape, B_2.shape)

[ 1, 8 ] [ 8, 2 ] [ 1, 2 ]


In [10]:
let h_2 = torch.add(torch.matmul(h_1, W_2), B_2);
h_2._name = "h_2"

"h_2"

In [11]:
h_2

Tensor {
  data: Float32Array(2) [ -85.49651336669922, 21.314876556396484 ],
  shape: [ 1, 2 ],
  requires_grad: true,
  _grad: undefined,
  _prev: [
    Tensor {
      data: Float32Array(2) [ -85.63175964355469, 21.314289093017578 ],
      shape: [ 1, 2 ],
      requires_grad: true,
      _grad: undefined,
      _prev: [
        Tensor {
          data: [Float32Array],
          shape: [Array],
          requires_grad: true,
          _grad: undefined,
          _prev: [Array],
          _name: "h_1",
          _backward: [Function (anonymous)]
        },
        Tensor {
          data: [Float32Array],
          shape: [Array],
          requires_grad: true,
          _grad: undefined,
          _prev: [],
          _name: "W_2",
          _backward: undefined
        }
      ],
      _name: undefined,
      _backward: [Function (anonymous)]
    },
    Tensor {
      data: Float32Array(2) [ 0.13524998724460602, 0.0005876394570805132 ],
      shape: [ 1, 2 ],
      requires_grad: true

In [12]:
logTree(h_2);

Gradient computation graph:
Tensor: Tensor(shape=[1,2]) (Name: h_2)
|  Prev Tensor: Tensor(shape=[1,2]) (Name: undefined), Grad: None
|  |  Prev Tensor: Tensor(shape=[1,8]) (Name: h_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: undefined), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[1,4]) (Name: Input (x)), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[4,8]) (Name: W_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: W_2), Grad: None
|  |  Prev Tensor: Tensor(shape=[8,2]) (Name: W_2), Grad: None
|  Prev Tensor: Tensor(shape=[1,2]) (Name: B_2), Grad: None


In [ ]:
h_2.backward(new torch.Tensor([0.1, 0.3], [1, 2]));

TypeError: Cannot read properties of undefined (reading 'length')

In [14]:
logTree(h_2);

Gradient computation graph:
Tensor: Tensor(shape=[1,2]) (Name: h_2)
|  Prev Tensor: Tensor(shape=[1,2]) (Name: undefined), Grad: 0.10000000149011612,0.30000001192092896
|  |  Prev Tensor: Tensor(shape=[1,8]) (Name: h_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: undefined), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[1,4]) (Name: Input (x)), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[4,8]) (Name: W_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: W_2), Grad: None
|  |  Prev Tensor: Tensor(shape=[8,2]) (Name: W_2), Grad: None
|  Prev Tensor: Tensor(shape=[1,2]) (Name: B_2), Grad: 0.10000000149011612,0.30000001192092896


In [15]:
// import * as nn from "https://raw.githubusercontent.com/etornam45/torch-ts/refs/heads/main/dist/bundle.ts"


In [16]:
import * as nn from "./index.ts";

class Model extends nn.Module {
  weights: nn.Tensor;
  bias: nn.Tensor;

  constructor(input_size: number, output_size: number) {
    super();
    this.weights = nn.Tensor.randn([input_size, output_size]);
    this.bias = nn.Tensor.randn([1, output_size]);
  }

  override forward(x: nn.Tensor) {
    return nn.add(nn.matmul(x, this.weights), this.bias);
  }

  override backward(grad: nn.Tensor) {
    this.weights._grad = nn.matmul(grad, this.weights.transpose());
    this.bias._grad = grad;
  }
}

const model = new Model(4, 2);
// const input = nn.Tensor.randn([1, 4]);
// const output = model.forward(input);
// output.backward(nn.Tensor.ones([1, 2]));
// model.weights._grad
// model.bias._grad